In [3]:
import coffea
import uproot
import pandas as pd

In [2]:
test_file = "./test_Spring2021.root"
file = uproot.open(test_file)
from coffea.nanoevents import NanoEventsFactory, FCC
events = NanoEventsFactory.from_root(
test_file+":events",
    entry_stop=100,
    schemaclass=FCC.get_schema(version="latest"),
    delayed=False,
    metadata=file["metadata"].arrays()
).events()
file.close()

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/schemas/fcc.py:4: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


### Added some more mixin classes and named all of them according to the edm4hep names (eg [link](https://github.com/key4hep/EDM4hep/blob/main/edm4hep.yaml#L253))

In [4]:
names = events.fields
mixins = [events[name].layout.content.parameter("__record__") for name in names]
collection_name = [events[name].layout.content.parameter("collection_name") for name in names]

df = pd.DataFrame(names)
df.columns = ["Field Name"]
df["Collection Name"] = collection_name
df["Mixin Class"] = mixins
df

,Field Name,Collection Name,Mixin Class
0,AllMuonidx0,AllMuonidx0,ObjectID
1,EFlowNeutralHadron,EFlowNeutralHadron,Cluster
2,EFlowNeutralHadron_0,None,None
3,EFlowNeutralHadron_1,None,None
4,EFlowNeutralHadron_2,None,None
5,EFlowNeutralHadronidx0,EFlowNeutralHadronidx0,ObjectID
6,EFlowNeutralHadronidx1,EFlowNeutralHadronidx1,ObjectID
7,EFlowNeutralHadronidx2,EFlowNeutralHadronidx2,ObjectID
8,EFlowPhoton,EFlowPhoton,Cluster
9,EFlowPhoton_0,None,None


#### These are the collections and their IDs. Seems like, they could be accessed exclusively by attaching the metadata to the events tree

In [5]:
df2 = pd.DataFrame()
df2["Collection Name"] = events.metadata.m_names[0]
df2["Collection ID"] = events.metadata.m_collectionIDs[0]
df2

,Collection Name,Collection ID
0,Electron,1
1,Muon,2
2,AllMuon,3
3,EFlowNeutralHadron,4
4,Particle,5
5,Photon,6
6,ReconstructedParticles,7
7,EFlowPhoton,8
8,MCRecoAssociations,9
9,MissingET,10


#### A problem: All the Tracks and Cluster branches are kind of empty, so cant deduce their behavior

In [9]:
events.EFlowNeutralHadron.iTheta #is empty

<Array [[], [], [], [], ..., [], [], [], []] type='100 * var * float32[para...'>

In [12]:
events.EFlowTrack.chi2 #is empty

<Array [[0, 0, 0, 0, 0, ..., 0, 0, 0, 0], ...] type='100 * var * float32[pa...'>

No idea what to do! Can't deduce their workings when they are empty. Also could not find any resources on them

#### Links to code:
- [Schema](https://github.com/prayagyadav/coffea/blob/fcc-schema/src/coffea/nanoevents/schemas/fcc.py)
- [Methods](https://github.com/prayagyadav/coffea/blob/fcc-schema/src/coffea/nanoevents/methods/fcc.py)